In [1]:
!pip install pyproj

In [2]:
import pandas as pd
from pyproj import Transformer

In [3]:
kunnat = pd.read_csv("data/kunnat/kunta4500k.csv")
kunnat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   FID     308 non-null    object
 1   kunta   308 non-null    int64 
 2   vuosi   308 non-null    int64 
 3   nimi    308 non-null    object
 4   namn    308 non-null    object
 5   name    308 non-null    object
 6   geom    308 non-null    object
dtypes: int64(2), object(5)
memory usage: 17.0+ KB


In [4]:
kunnat_dict = kunnat.to_dict("list")
temp = {}
structure = ["FID", "kunta", "vuosi", "nimi", "namn", "name", "coordinates"]
si = 0

for column_name in kunnat_dict:
    for i, k in enumerate(kunnat_dict[column_name]):
        if i in temp:
            temp[i].update({structure[si]: k})
        else:
            temp[i] = {structure[si]: k}
    si += 1


In [10]:
import re

features = []

def handle_polygon(p):
    result = []
    search = re.compile(r"(\d+.\d+) (\d+.\d+)")
    l = search.findall(p)
    transformer = Transformer.from_crs("EPSG:3067", "EPSG:4326")
    
    for pair in l:
        coordinates = transformer.transform(float(pair[0]), float(pair[1]))
        result.append(list(coordinates))
        
    return result
    

    
    

# polygon = "POLYGON ((366787.92400007433 7001300.582998411, 362458.797000109 6995675.228997898, 363260.7930000954 6987161.697998049, 357374.74200016126 6990194.168997132, 352599.11600023136 6984860.2019962305, 349403.0040002923 6985932.9889954645, 346951.9410003302 6973669.695994952, 354508.0040001854 6970011.823996754, 348733.1870002747 6959704.550995577, 343298.20400040667 6961570.194994021, 339046.7090005409 6963078.848992498, 323281.48600148177 6981415.651983124, 321987.07200160914 6984871.923981947, 322692.6630015667 6988079.051982387, 334329.4410008429 7002293.675989546, 335875.88900075614 6998150.952990417, 357452.6040001728 7005219.799997036, 366787.92400007433 7001300.582998411))"

# print(handle_polygon(polygon))


for k, v in temp.items():
    result = { "type": "Feature",
              "geometry": {
                  "type": "Polygon",
                  "coordinates": [handle_polygon(v["coordinates"])]
              },
              "properties": {
                  "id": v["kunta"],
                  "name": v["nimi"]
              }
             }
    features.append(result)

print(features[0])

geojson = {"type": "FeatureCollection", "features": features}


{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[63.11642950937275, 24.359007626173202], [63.06437535380347, 24.278016678975813], [62.98835315277282, 24.300936607202797], [63.01327363142358, 24.18229643786593], [62.96355243645589, 24.092852576241295], [62.97185820361599, 24.028940138920387], [62.860918645461396, 23.99196412954902], [62.831217447644356, 24.143433221373133], [62.73648419728666, 24.03952144927343], [62.75092581364169, 23.931593904754042], [62.76260589820092, 23.847048841670077], [62.91962829590023, 23.519471162088397], [62.94996119653581, 23.490324658143237], [62.97903719389498, 23.5007685875876], [63.111901184558164, 23.71563834943641], [63.07548536125099, 23.75038702635897], [63.1479983417847, 24.170771051142268], [63.11642950937275, 24.359007626173202]]]}, 'properties': {'id': 5, 'name': 'Alajärvi'}}


In [12]:
import json
with open("./data/test.json", "w") as fp:
    json.dump(geojson, fp)

In [7]:
from pyproj import Transformer


